it was pretrained with three objectives:

* Distillation loss: the model was trained to return the same probabilities as the BERT base model.
* Masked language modeling (MLM): this is part of the original training loss of the BERT base model. When taking a sentence, the model randomly masks 15% of the words in the input then run the entire masked sentence through the model and has to predict the masked words. This is different from traditional recurrent neural networks (RNNs) that usually see the words one after the other, or from autoregressive models like GPT which internally mask the future tokens. It allows the model to learn a bidirectional representation of the sentence.
* Cosine embedding loss: the model was also trained to generate hidden states as close as possible as the BERT base model.

In [4]:
from transformers import DistilBertTokenizer, DistilBertModel

In [5]:
'''
主要预训练模型:
'distilbert-base-uncased'
'distilbert-base-cased'
'distilroberta-base'
'''
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
print(tokenizer.model_input_names)
print(tokenizer.all_special_tokens)

['input_ids', 'attention_mask']
['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [6]:
model = DistilBertModel.from_pretrained("bert-base-uncased")
model

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing DistilBertModel: ['bert.encoder.layer.1.output.dense.weight', 'bert.encoder.layer.3.intermediate.dense.weight', 'bert.encoder.layer.6.attention.output.LayerNorm.weight', 'bert.encoder.layer.6.intermediate.dense.weight', 'bert.encoder.layer.9.intermediate.dense.weight', 'bert.encoder.layer.7.output.dense.bias', 'bert.encoder.layer.8.attention.output.dense.weight', 'bert.encoder.layer.6.attention.self.value.weight', 'cls.predictions.transform.dense.bias', 'bert.encoder.layer.9.attention.output.dense.weight', 'bert.encoder.layer.4.output.dense.bias', 'bert.encoder.layer.2.intermediate.dense.weight', 'bert.encoder.layer.8.attention.output.dense.bias', 'bert.encoder.layer.9.intermediate.dense.bias', 'bert.encoder.layer.2.attention.self.value.weight',

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [7]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
print(encoded_input)
print(encoded_input.keys())
print(tokenizer.decode(encoded_input['input_ids'].tolist()[0]))

{'input_ids': tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
dict_keys(['input_ids', 'attention_mask'])
[CLS] replace me by any text you'd like. [SEP]


In [9]:
result = model(**encoded_input)
print(result.last_hidden_state.shape)
print(result.last_hidden_state)



torch.Size([1, 12, 768])
tensor([[[-0.4749, -1.4572, -0.6376,  ...,  0.9180, -0.6711,  0.1183],
         [-2.9043, -0.0772, -0.8075,  ...,  0.6107,  0.7496, -0.4124],
         [-2.2385, -1.2789, -1.7823,  ...,  0.9904, -0.8160,  0.2360],
         ...,
         [-0.8106, -1.8260, -0.7633,  ...,  0.8127, -0.3572, -0.3430],
         [-1.2895, -0.2100, -1.7359,  ...,  1.2879,  0.3218, -0.8977],
         [-2.4373, -1.7458, -1.0812,  ...,  0.3782,  0.2822,  1.3520]]],
       grad_fn=<NativeLayerNormBackward0>)
